In [1]:
#Importing libraries.

import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import pyautogui
import shutup
import requests
from bs4 import BeautifulSoup
shutup.please()

In [8]:
link = 'https://www.eltiempo.es/catalunya'

r = requests.get(link)

In [9]:
driver = webdriver.Chrome()
driver.get(link)

In [38]:
provinces = driver.find_elements(By.XPATH,  "//a[contains(@href, 'en-provincia')]")

In [39]:
provinces[1].get_attribute('href')

'https://www.eltiempo.es/en-provincia-girona'

In [40]:
for i in provinces:
    print(i.get_attribute('href'))

https://www.eltiempo.es/en-provincia-barcelona
https://www.eltiempo.es/en-provincia-girona
https://www.eltiempo.es/en-provincia-lleida
https://www.eltiempo.es/en-provincia-tarragona


In [26]:
for i in provinces:
    try:
        print(i.get_attribute('href'))
    except:
        pass